In [47]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
from scipy.io import arff
import pandas as pd
from time import time

In [48]:
# load data
df=pd.read_csv('ALOI_norm.csv')  

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att19,att20,att21,att22,att23,att24,att25,att26,att27,outlier
0,0.784999,0.000018,0.0,0.000093,0.0,0.0,0.0,0.0,0.0,0.139811,...,0.048171,0.001189,0.0,0.115728,0.023449,0.0002,0.000220,0.034952,0.046914,1
1,0.958088,0.000000,0.0,0.001671,0.0,0.0,0.0,0.0,0.0,0.019556,...,0.000000,0.000000,0.0,0.022035,0.007516,0.0000,0.000000,0.001033,0.000000,1
2,0.938768,0.000000,0.0,0.005146,0.0,0.0,0.0,0.0,0.0,0.018451,...,0.000000,0.000000,0.0,0.035542,0.011982,0.0000,0.001595,0.019520,0.000000,1
3,0.954775,0.000000,0.0,0.001427,0.0,0.0,0.0,0.0,0.0,0.024944,...,0.000000,0.000000,0.0,0.019941,0.000805,0.0000,0.000000,0.000035,0.000000,1
4,0.933601,0.000000,0.0,0.001682,0.0,0.0,0.0,0.0,0.0,0.037002,...,0.000000,0.000000,0.0,0.046759,0.002663,0.0000,0.000339,0.001359,0.000000,1


In [49]:
df.shape

(50000, 28)

In [50]:
# data to values
df = df.values

In [51]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/aloi.json")
gc = GCForest(config)

In [52]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,27]
X = df[:,0:27]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [53]:
# number of class
len(np.unique(y_train))

2

In [54]:
         # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
   

[ 2018-04-26 14:27:40,032][cascade_classifier.fit_transform] X_groups_train.shape=[(33500, 27)],y_train.shape=(33500,),X_groups_test.shape=[(16500, 27)],y_test.shape=(16500,)
[ 2018-04-26 14:27:40,035][cascade_classifier.fit_transform] group_dims=[27]
[ 2018-04-26 14:27:40,036][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 14:27:40,037][cascade_classifier.fit_transform] group_ends=[27]
[ 2018-04-26 14:27:40,039][cascade_classifier.fit_transform] X_train.shape=(33500, 27),X_test.shape=(16500, 27)
[ 2018-04-26 14:27:40,043][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(33500, 27), X_cur_test.shape=(16500, 27)
[ 2018-04-26 14:27:40,898][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.37%
[ 2018-04-26 14:27:42,057][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.37%
[ 2018-04-26 14:27:43,156][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estim

In [55]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-26 14:27:50,989][cascade_classifier.transform] X_groups_test.shape=[(16500, 27)]
[ 2018-04-26 14:27:50,991][cascade_classifier.transform] group_dims=[27]
[ 2018-04-26 14:27:50,992][cascade_classifier.transform] X_test.shape=(16500, 27)
[ 2018-04-26 14:27:50,995][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(16500, 27)


Test Accuracy of ExtraTrees = 97.042424 %


In [56]:
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-26 14:27:53,396][cascade_classifier.transform] X_groups_test.shape=[(16500, 27)]
[ 2018-04-26 14:27:53,398][cascade_classifier.transform] group_dims=[27]
[ 2018-04-26 14:27:53,399][cascade_classifier.transform] X_test.shape=(16500, 27)
[ 2018-04-26 14:27:53,402][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(16500, 27)


In [57]:
# New data
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-26 14:27:55,945][cascade_classifier.fit_transform] X_groups_train.shape=[(33500, 29)],y_train.shape=(33500,),X_groups_test.shape=[(16500, 29)],y_test.shape=(16500,)
[ 2018-04-26 14:27:55,948][cascade_classifier.fit_transform] group_dims=[29]
[ 2018-04-26 14:27:55,950][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 14:27:55,951][cascade_classifier.fit_transform] group_ends=[29]
[ 2018-04-26 14:27:55,952][cascade_classifier.fit_transform] X_train.shape=(33500, 29),X_test.shape=(16500, 29)
[ 2018-04-26 14:27:55,959][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(33500, 29), X_cur_test.shape=(16500, 29)
[ 2018-04-26 14:27:56,791][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.88%
[ 2018-04-26 14:27:57,841][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.79%
[ 2018-04-26 14:27:58,898][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estim

In [58]:
# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-26 14:28:07,013][cascade_classifier.transform] X_groups_test.shape=[(16500, 29)]
[ 2018-04-26 14:28:07,015][cascade_classifier.transform] group_dims=[29]
[ 2018-04-26 14:28:07,017][cascade_classifier.transform] X_test.shape=(16500, 29)
[ 2018-04-26 14:28:07,019][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(16500, 29)


Test Accuracy of ExtraTrees = 97.733333 %


In [59]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100),i,'fois')

In [60]:
# Deep
for i in range (20):
    t0 = time()
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    tt = time() - t0
    affiche() 
    print (" Time ",format(round(tt,3))," seconds")

[ 2018-04-26 14:28:09,545][cascade_classifier.fit_transform] X_groups_train.shape=[(33500, 29)],y_train.shape=(33500,),X_groups_test.shape=[(16500, 29)],y_test.shape=(16500,)
[ 2018-04-26 14:28:09,549][cascade_classifier.fit_transform] group_dims=[29]
[ 2018-04-26 14:28:09,550][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 14:28:09,551][cascade_classifier.fit_transform] group_ends=[29]
[ 2018-04-26 14:28:09,553][cascade_classifier.fit_transform] X_train.shape=(33500, 29),X_test.shape=(16500, 29)
[ 2018-04-26 14:28:09,557][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(33500, 29), X_cur_test.shape=(16500, 29)
[ 2018-04-26 14:28:10,334][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.82%
[ 2018-04-26 14:28:11,426][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.82%
[ 2018-04-26 14:28:12,511][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estim

('Test Accuracy of ExtraTrees = 97.769697 %', 0, 'fois')
(' Time ', '11.009', ' seconds')


[ 2018-04-26 14:28:23,956][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.61%
[ 2018-04-26 14:28:25,031][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.82%
[ 2018-04-26 14:28:26,112][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.06%
[ 2018-04-26 14:28:27,186][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.73%
[ 2018-04-26 14:28:28,449][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.73%
[ 2018-04-26 14:28:29,499][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.82%
[ 2018-04-26 14:28:30,590][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.70%
[ 2018-04-26 14:28:31,629][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.64%


('Test Accuracy of ExtraTrees = 97.660606 %', 1, 'fois')
(' Time ', '11.087', ' seconds')


[ 2018-04-26 14:28:37,817][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.67%
[ 2018-04-26 14:28:38,866][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.73%
[ 2018-04-26 14:28:39,986][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.18%
[ 2018-04-26 14:28:41,044][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.06%
[ 2018-04-26 14:28:42,240][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.73%
[ 2018-04-26 14:28:43,290][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.91%
[ 2018-04-26 14:28:44,347][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.73%
[ 2018-04-26 14:28:45,408][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.70%


...

('Test Accuracy of ExtraTrees = 97.242424 %', 17, 'fois')
(' Time ', '11.118', ' seconds')


[ 2018-04-26 14:32:12,433][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.37%
[ 2018-04-26 14:32:13,485][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.49%
[ 2018-04-26 14:32:14,541][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.52%
[ 2018-04-26 14:32:15,566][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.67%
[ 2018-04-26 14:32:16,709][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.46%
[ 2018-04-26 14:32:17,774][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.46%
[ 2018-04-26 14:32:18,862][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.61%
[ 2018-04-26 14:32:19,962][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.40%


('Test Accuracy of ExtraTrees = 97.224242 %', 18, 'fois')
(' Time ', '11.004', ' seconds')


[ 2018-04-26 14:32:25,955][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.28%
[ 2018-04-26 14:32:27,168][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.40%
[ 2018-04-26 14:32:28,287][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.52%
[ 2018-04-26 14:32:29,332][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.61%
[ 2018-04-26 14:32:30,422][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.52%
[ 2018-04-26 14:32:31,562][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.46%
[ 2018-04-26 14:32:32,667][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.49%
[ 2018-04-26 14:32:33,716][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.58%


('Test Accuracy of ExtraTrees = 97.206061 %', 19, 'fois')
(' Time ', '11.261', ' seconds')


# ('Test Accuracy of ExtraTrees = 97.769697 %', 0, 'fois') 


#  ('Compte fait en ', '11.009', ' seconds')